<a href="https://colab.research.google.com/github/7rohj/.-/blob/main/%EC%B2%A0%EB%8F%84(for).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

year = [2019,2025,2030]

for q in year:
  # 구글 드라이브 마운트, 일일이 파일 업로드할 필요 없음
  # 마지막 / 이후 파일명 수정하면 됨
  train_info2019 = pd.read_table(f'/content/drive/MyDrive/철도/{q}_TRANSIT_INFO.txt', sep='\t', encoding='euc-kr')
  train_path2019 = pd.read_table(f'/content/drive/MyDrive/철도/{q}_TRANSIT_PATH.txt', sep='\t', encoding='euc-kr')
  train_info2019.head(3)

  # 도로 = 3, 철도 = 4/ 철도만 추출
  train_info2019 = train_info2019[train_info2019['Mode'] == 4]
  train_info2019 = train_info2019.reset_index(drop=True)
  train_path2019['Station_Y/N'] = train_path2019['Station_Y/N'].replace(1,0.75)
  train_path2019['Station_Y/N'] = train_path2019['Station_Y/N'].replace(0,'#.00')
  train_info2019.tail(3)

  dataframe = pd.DataFrame(None)
  blist = []

  for i in range(len(train_info2019)):
    alist = [] # 한 줄 씩 여러개 리스트 생성해서 blist 에 append
    alist.append('a')
    alist.append(train_info2019['Line_ID'].tolist()[i])
    alist.append('|')
    alist.append('2')
    alist.append(train_info2019['Peak_Headway'].tolist()[i])
    alist.append('0.01')
    alist.append(train_info2019['Name'].tolist()[i])
    alist.append(' ') # UT1
    alist.append(' ') # UT2
    alist.append(' ') # UT3
    blist.append(alist)

  # ~ 여기까지 1단계 완료
  # 아래는 보기 편하게 테이블 형식으로 출력

  def convert_nested_to_str(lst):
      return [[str(x) if isinstance(x, int) else x for x in sublist] for sublist in lst]

  converted_list = convert_nested_to_str(blist)
  converted_list

  dataframe = pd.DataFrame(blist)

  # ADD/ LINE/ MODE/ VEH/ HEADWAY/ SPEED/ DESCR/ UT1/ UT2/ UT3
  dataframe.rename(columns={0:'ADD', 1:"LINE", 2:"MODE", 3:"VEH", 4:"HEADWAY",
                            5:"SPEED", 6:"DESCR", 7:"UT1", 8:"UT2", 9:"UT3"}, inplace=True)
  dataframe['ETC'] = converted_list
  dataframe['ETC'] = dataframe['ETC'].apply(lambda x: ', '.join(map(str, x))).str.replace('[', '').str.replace(']', '').str.replace("'", '') # 이중리스트 [] 제거..
  dataframe['ETC'] = dataframe['ETC'].apply(lambda x: ''.join(map(str, x)).replace(',', '')) # , 를  로 변경
  dataframe

  #idea.. 이중 리스트 해서 안의 리스트 원소 개수로 계산

  seq_nodeid_lst = []
  seq_sttyn_lst = []
  lineid = []

  for i in range(1,161):

    num = dataframe['LINE'].tolist()[i-1]
    num_str = str(num)

    # append X > extend
    seq_sttyn_lst.extend(train_path2019[train_path2019['Line_ID'] == num]['Station_Y/N'].tolist())
    seq_nodeid_lst.extend(train_path2019[train_path2019['Line_ID'] == num]['Node_id'].tolist())
    tmp = [i] * len(train_path2019[train_path2019['Line_ID'] == num]['Station_Y/N'].tolist())
    lineid.append(tmp)

    #print(i,'번째, ',len(seq_sttyn_lst),len(seq_nodeid_lst))

  lineid = [item for sublist in lineid for item in sublist]
  new_df = pd.DataFrame({'column0': [f"dwt={seq_sttyn_lst[i]} {seq_nodeid_lst[i]}" for i in range(len(seq_sttyn_lst))]})
  new_df['LINE_ID'] = lineid
  new_df['LINE_ID'] = new_df['LINE_ID'].astype(int)
  new_df

  final = []

  for s in range(len(dataframe)):

    #tmp = []

    final.append(dataframe['ETC'][s])
    final.append('ttf=1')
    final.append('path=no')
    final.extend(new_df[new_df['LINE_ID'] == s+1]['column0'].reset_index(drop=True).values)

    #final.append(tmp)

    pd.DataFrame(final).to_excel(f'/content/drive/MyDrive/철도/Transit Data(Line_Input)_{q}_완료.xlsx')

<ipython-input-4-42e6e85f4992>:52: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe['ETC'] = dataframe['ETC'].apply(lambda x: ', '.join(map(str, x))).str.replace('[', '').str.replace(']', '').str.replace("'", '') # 이중리스트 [] 제거..
